In [1]:
#
import sys
import os
sys.path.append("/home/sergey/drclinics/common")
sys.path.append("/home/sergey/drclinics/reports")
#


import pandas as pd
from report_utils import query_by_name, send_mail
from configs import load_json_config

from connect_platform import connect_telemed
from log import log
import datetime
from utils import get_path, DATETIME_FORMAT

from universal_connection import UniversalConnection, DBType

from log import log

In [46]:
sql = """
with t1 as (
select 	row_number() over (partition by patient.id order by end_ desc) rownum,
		patient.id patient_id,
		concat_ws(' ', p_patient.last_name, p_patient.first_name, p_patient.middle_name) patient_fio,
		case when pc.product_id in (
            select id from product
            where full_name like '%ВЭБ%'
        ) then 'VEBMED'
        else 'ПРОЧИЕ'
        end promo_type,
        --partner."name" partner_name,
        --promo.start_,
        promo.end_
from patient
--
inner join person p_patient on p_patient.id = patient.person_id
--
inner join patient_promotion as pp on pp.patient_id = patient.id
inner join promotion promo on promo.id = pp.promotion_id 
--
left join product_condition pc
   		on promo.product_condition_id = pc.id
--
inner join appointment a on a.patient_id = patient.id 
--
inner join partner on promo.partner_id = partner.id 
--
where pc.product_id in (
            	select id from product
            	where full_name like '%ВЭБ%')
-----
),
-----
t1_group as (
select		patient_id,
			patient_fio,
			promo_type,
			end_
from t1 
--
where rownum = 1
),
t2 as (
select 	row_number() over (partition by patient.id order by end_ desc) rownum,
		patient.id patient_id,
		concat_ws(' ', p_patient.last_name, p_patient.first_name, p_patient.middle_name) patient_fio,
        partner."name" partner_name,
        --promo.start_,
        promo.end_
from patient
--
inner join person p_patient on p_patient.id = patient.person_id
--
inner join patient_promotion as pp on pp.patient_id = patient.id
inner join promotion promo on promo.id = pp.promotion_id 
--
left join product_condition pc
   		on promo.product_condition_id = pc.id
--
inner join appointment a on a.patient_id = patient.id 
--
inner join partner on promo.partner_id = partner.id 
--
where pc.product_id not in (
            	select id from product
            	where full_name like '%ВЭБ%')
-----
),
-----
t2_group as (
select		patient_id,
			patient_fio,
			partner_name,
			end_
from t2 
--
where rownum = 1
and end_ > '2020-05-31'
)
-----------------------------
select 	t1_group.patient_id,
		t1_group.patient_fio,
		t1_group.promo_type,
		t1_group.end_,
		t2_group.partner_name,
		t2_group.end_ partner_end
from t1_group left join t2_group 
on t1_group.patient_id = t2_group.patient_id
"""

In [47]:
connection = UniversalConnection('../../../.credentials/telemed/prom.cfg', DBType.Postgres)
data = connection.query(sql)
connection.close()

2020-05-19 11:32:09 connect to postgres database using config file "../../../.credentials/telemed/prom.cfg"
2020-05-19 11:32:09 creating ssh tunnel to 172.16.100.19 as root...
2020-05-19 11:32:09 connect postgres using parameters:
                    database: telemed
				    user: norekhov
				    password: ***masked***
				    host: localhost
				    port: 44409
2020-05-19 11:32:09 @telemed: execute sql:
				    SET TIME ZONE 'Europe/Moscow'
				    None
2020-05-19 11:32:10 @telemed query:
                    with t1 as (
				    select 	row_number() over (partition by patient.id order by end_ desc) rownum,
				    		patient.id patient_id,
				    		concat_ws(' ', p_patient.last_name, p_patient.first_name, p_patient.middle_name) patient_fio,
				    		case when pc.product_id in (
				                select id from product
				                where full_name like '%ВЭБ%'
				            ) then 'VEBMED'
				            else 'ПРОЧИЕ'
				            end promo_type,
				            --par

In [48]:
df = pd.DataFrame(data).copy()

In [49]:
df.head(15)

,patient_id,patient_fio,promo_type,end_,partner_name,partner_end
0,963,Зайцев Сергей Львович,VEBMED,2020-05-31,Ренессанс Страхование,2020-12-02
1,1425,Чернин Максим Борисович,VEBMED,2020-05-31,Ренессанс Страхование,2020-12-02
2,4386,Швецов Денис Евгеньевич,VEBMED,2020-05-31,Доктор рядом,2021-03-27
3,4984,Коньшин Александр Вадимович,VEBMED,2020-05-31,Альянс Жизнь,2021-04-30
4,7941,Шокель Антон Владимирович,VEBMED,2020-05-31,DocDoc,2021-04-29
5,8025,Силкина Елена Владимировна,VEBMED,2020-05-31,Ренессанс Страхование,2020-12-02
6,10725,Знаменский Иван Сергеевич,VEBMED,2020-05-31,СК Согласие,2020-08-19
7,12294,Геращенко Анна Петровна,VEBMED,2020-05-31,Доктор рядом,2021-02-05
8,12647,Гавриленко Ксения Михайловна,VEBMED,2020-05-31,Ренессанс Страхование,2021-12-31
9,13711,Клименко Алёна Владимировна,VEBMED,2020-05-31,DocDoc,2021-06-18


In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22409 entries, 0 to 22408
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   patient_id    22409 non-null  int64 
 1   patient_fio   22409 non-null  object
 2   promo_type    22409 non-null  object
 3   end_          22409 non-null  object
 4   partner_name  984 non-null    object
 5   partner_end   984 non-null    object
dtypes: int64(1), object(5)
memory usage: 1.0+ MB


In [52]:
# web-patients
len(df.patient_id.unique())

22409

In [64]:
df['partner_name'].isnull().values.sum()

21425

In [65]:
len(df.patient_id.unique()) - df['partner_name'].isnull().values.sum()

984

In [66]:
d = dict()

d = {
    'Всего пациентов WEB':len(df.patient_id.unique()),
    'Пациентов с другим продуктом':len(df.patient_id.unique()) - df['partner_name'].isnull().values.sum()
    
}

In [69]:
report = pd.DataFrame.from_dict(d, orient='index').sort_index()
report.columns = ['Количество']

In [70]:
report

,Количество
Всего пациентов WEB,22409
Пациентов с другим продуктом,984


In [71]:
report_date: str = datetime.datetime.now().strftime(DATETIME_FORMAT).replace(':', '-')

In [81]:
file_name = 'after_web' + report_date + '.xlsx'
writer = pd.ExcelWriter(file_name, engine='xlsxwriter')

df.to_excel(writer, sheet_name='Data', index=False)
worksheet = writer.sheets['Data']
# worksheet.set_column('A:A', 50)
worksheet.set_column(0, 0, 12)
worksheet.set_column(1, 1, 35)
worksheet.set_column(2, 3, 12)
worksheet.set_column(4, 4, 25)
worksheet.set_column(5, 5, 12)

report.to_excel(writer, sheet_name='Count')
worksheet = writer.sheets['Count']
worksheet.set_column('A:A', 35)
worksheet.set_column(1, len(report.columns), 12)


writer.save()